###Import Libraries and set directory



In [19]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.2.1
  # !pip install stanfordcorenlp
  # !pip install git+https://github.com/leonardozago/pybabelfy.git
  # !pip install levenshtein
  # !pip install scispacy
  # !pip install contractions
  # !pip install unidecode

In [20]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version('1.2.1')
except AttributeError:
    raise ValueError(
        f'This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>.'
    ) from None

In [21]:
import networkx as nx
import pandas as pd
import numpy as np
import os
import random
import shutil
import requests
import gzip
# import unidecode
import csv
import re

import subprocess
from subprocess import Popen

import stellargraph as sg
from stellargraph.data import UnsupervisedSampler
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification

from tensorflow import keras

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import OneClassSVM
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Convert KGen generated Triples to DBLP data

Choosing data samples directory

In [23]:
data_dir_triples = '/content/drive/MyDrive/TCC_Leonardo_Zago/data_samples/triples'
unstructured_reviews_dir = '/content/drive/MyDrive/TCC_Leonardo_Zago/data_samples/unstructured_reviews'

Cleansing data from triples extraction

In [24]:
triples_df = pd.read_csv(
    os.path.join(data_dir_triples, 'review_Video_Games_preprocessed_triples.txt'),
    sep='\t',
    header=None,
    names=['vertex', 'source', 'edge', 'target'],
)

triples_df = triples_df.dropna()
source_list = list(pd.concat([triples_df['source'], triples_df['target']]).drop_duplicates())

triples_df['source_index'] = triples_df.apply(lambda row: source_list.index(row['source']), axis=1)
triples_df['target_index'] = triples_df.apply(lambda row: source_list.index(row['target']), axis=1)
triples_df = triples_df[['source_index', 'target_index', 'vertex', 'source', 'edge', 'target']].sort_values('source_index').reset_index(drop=True)

display(triples_df.head())

,source_index,target_index,vertex,source,edge,target
0,0,20,s2236,it nhl 14 with bergeron on the box,rdf:subject,it
1,0,4267,s2236,it nhl 14 with bergeron on the box,rdf:predicate,nhl
2,0,4268,s2236,it nhl 14 with bergeron on the box,rdf:object,14
3,0,4269,s2236,it nhl 14 with bergeron on the box,local:AM-LOC,box
4,0,4270,s2236,it nhl 14 with bergeron on the box,local:AM-MNR,bergeron


Get number of nodes in KGen graph

In [25]:
n_nodes = len(source_list)
n_nodes

6086

Applying CountVectorizer() to convert text sources to a matrix of token counts

In [26]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(source_list) # Ao invés de source_list passar uma lista com todas as reviews selecionadas após o tratamento
vectorizer.get_feature_names_out()

vectors = X.toarray()

feature_names = ['w_{}'.format(ii) for ii in range(len(vectors[0]))]
node_data = pd.DataFrame(vectors, columns=feature_names)

display(node_data.head())

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_2190,w_2191,w_2192,w_2193,w_2194,w_2195,w_2196,w_2197,w_2198,w_2199
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Getting edgelist with the relations between text sources and targets

In [27]:
edgelist = triples_df[['source_index', 'target_index']].rename(columns={'source_index': 'source', 'target_index': 'target'})
edgelist['label'] = 'cites'  # set the edge type

display(edgelist.head())

,source,target,label
0,0,20,cites
1,0,4267,cites
2,0,4268,cites
3,0,4269,cites
4,0,4270,cites


Get number of edges in KGen graph

In [28]:
print(edgelist.shape[0])

11806



## Link prediction via inductive node representations with attri2vec

This implementation of attri2vec below can be found on https://stellargraph.readthedocs.io/en/stable/demos/link-prediction/attri2vec-link-prediction.html

This demo notebook demonstrates how to perform link prediction for out-of-sample nodes through learning node representations inductively with attri2vec [1]. The implementation uses the stellargraph components.

<a name='refs'></a>
**References:** 

[1] [Attributed Network Embedding via Subspace Discovery](https://link.springer.com/article/10.1007/s10618-019-00650-2). D. Zhang, Y. Jie, X. Zhu and C. Zhang. Data Mining and Knowledge Discovery, 2019. 

### attri2vec

attri2vec learns node representations by performing a linear/non-linear mapping on node content attributes. To make the learned node representations respect structural similarity, [DeepWalk](https://dl.acm.org/citation.cfm?id=2623732)/[Node2Vec](https://snap.stanford.edu/node2vec) learning mechanism is used to make nodes sharing similar random walk context nodes represented closely in the subspace, which is achieved by maximizing the occurrence probability of context nodes conditioned on the representation of the target nodes. 

In this demo, we first train the attri2vec model on the in-sample subgraph and obtain a mapping function from node attributes to node representations, then apply the mapping function to the content attributes of out-of-sample nodes and obtain the representations of out-of-sample nodes. We evaluate the quality of inferred out-of-sample node representations by using it to predict the links of out-of-sample nodes.

### Loading DBLP network data

Construct the whole graph from edge list.

In [29]:
G_all_nx = nx.from_pandas_edgelist(edgelist, edge_attr='label')

Specify node types.

In [30]:
nx.set_node_attributes(G_all_nx, 'paper', 'label')

Get node features.

In [31]:
all_node_features = node_data[feature_names]

Create the Stellargraph with node features.

In [32]:
G_all = sg.StellarGraph.from_networkx(G_all_nx, node_features=all_node_features)

In [33]:
print(G_all.info())

StellarGraph: Undirected multigraph
 Nodes: 6086, Edges: 9406

 Node types:
  paper: [6086]
    Features: float32 vector, length 2200
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [9406]
        Weights: all 1 (default)
        Features: none


### Get DBLP Subgraph 
#### with papers published before a threshold year

Get the edge list connecting in-sample nodes.

In [34]:
subgraph_edgelist = edgelist.sample(frac=0.3).reset_index(drop=True) # Get a subgraph from an edgelist sample
display(subgraph_edgelist.head())

,source,target,label
0,793,4319,cites
1,1831,518,cites
2,684,4758,cites
3,1013,4922,cites
4,4220,4311,cites


Construct the network from the selected edge list.

In [35]:
G_sub_nx = nx.from_pandas_edgelist(subgraph_edgelist, edge_attr='label')

Specify node types.

In [36]:
nx.set_node_attributes(G_sub_nx, 'paper', 'label')

Get the ids of the nodes in the selected subgraph.

In [37]:
subgraph_node_ids = sorted(list(G_sub_nx.nodes))

Get the node features of the selected subgraph.

In [38]:
subgraph_node_features = node_data[feature_names].reindex(subgraph_node_ids)

Create the Stellargraph with node features.

In [39]:
G_sub = sg.StellarGraph.from_networkx(G_sub_nx, node_features=subgraph_node_features)

In [40]:
print(G_sub.info())

StellarGraph: Undirected multigraph
 Nodes: 3610, Edges: 3138

 Node types:
  paper: [3610]
    Features: float32 vector, length 2200
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [3138]
        Weights: all 1 (default)
        Features: none


### Train attri2vec on the DBLP Subgraph

Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk.

In [41]:
nodes = list(G_sub.nodes())
number_of_walks = 2
length = 5

Create the UnsupervisedSampler instance with the relevant parameters passed to it.

In [42]:
unsupervised_samples = UnsupervisedSampler(
    G_sub, nodes=nodes, length=length, number_of_walks=number_of_walks
)

Set the batch size and the number of epochs. 

In [43]:
batch_size = 50
epochs = 6

Define an attri2vec training generator, which generates a batch of (feature of target node, index of context node, label of node pair) pairs per iteration.

In [44]:
generator = Attri2VecLinkGenerator(G_sub, batch_size)

Building the model: a 1-hidden-layer node representation ('input embedding') of the `target` node and the parameter vector ('output embedding') for predicting the existence of `context node` for each `(target context)` pair, with a link classification layer performed on the dot product of the 'input embedding' of the `target` node and the 'output embedding' of the `context` node.

attri2vec part of the model, with a 128-dimension hidden layer, no bias term and no normalization. (Normalization can be set to 'l2'). 

In [45]:
layer_sizes = [128]
attri2vec = Attri2Vec(
    layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None
)

In [46]:
# Build the model and expose input and output sockets of attri2vec, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

Use the link_classification function to generate the prediction, with the `ip` edge embedding generation method and the `sigmoid` activation, which actually performs the dot product of the 'input embedding' of the target node and the 'output embedding' of the context node followed by a sigmoid activation. 

In [47]:
prediction = link_classification(
    output_dim=1, output_act='sigmoid', edge_embedding_method='ip'
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


Stack the attri2vec encoder and prediction layer into a Keras model, and specify the loss.

In [48]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-2),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Train the model.

In [49]:
history = model.fit(
    generator.flow(unsupervised_samples),
    epochs=epochs,
    verbose=2,
    use_multiprocessing=False,
    workers=1,
    shuffle=True,
)

Epoch 1/6
1156/1156 - 17s - loss: 0.9476 - binary_accuracy: 0.5183 - 17s/epoch - 14ms/step
Epoch 2/6
1156/1156 - 15s - loss: 0.7349 - binary_accuracy: 0.5991 - 15s/epoch - 13ms/step
Epoch 3/6
1156/1156 - 10s - loss: 0.5114 - binary_accuracy: 0.7607 - 10s/epoch - 8ms/step
Epoch 4/6
1156/1156 - 9s - loss: 0.3075 - binary_accuracy: 0.8781 - 9s/epoch - 8ms/step
Epoch 5/6
1156/1156 - 10s - loss: 0.1922 - binary_accuracy: 0.9320 - 10s/epoch - 8ms/step
Epoch 6/6
1156/1156 - 10s - loss: 0.1412 - binary_accuracy: 0.9511 - 10s/epoch - 8ms/step


### Predicting links of out-of-sample nodes with the learned attri2vec model

Build the node based model for predicting node representations from node content attributes with the learned parameters. Below a Keras model is constructed, with `x_inp[0]` as input and `x_out[0]` as output. Note that this model's weights are the same as those of the corresponding node encoder in the previously trained node pair classifier.

In [50]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

Get the node embeddings, for both in-sample and out-of-sample nodes, by applying the learned mapping function to node content features.

In [51]:
node_ids = node_data.index
node_gen = Attri2VecNodeGenerator(G_all, batch_size).flow(node_ids)
node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

122/122 [==============================] - 1s 4ms/step


Get the positive and negative edges for in-sample nodes and out-of-sample nodes. The edges of the in-sample nodes only include the edges between in-sample nodes, and the edges of out-of-sample nodes are referred to all the edges linked to out-of-sample nodes, including the edges connecting in-sample and out-of-sample edges.

In [52]:
in_sample_edges = []
out_of_sample_edges = []

in_sample_edges_df = subgraph_edgelist
out_of_sample_edges_df = pd.concat([edgelist, in_sample_edges_df]).drop_duplicates(keep=False).reset_index(drop=True)

for ii in range(len(in_sample_edges_df)):
    source_index = in_sample_edges_df['source'][ii]
    target_index = in_sample_edges_df['target'][ii]
    if source_index > target_index:  # neglect edge direction for the undirected graph
        continue
    in_sample_edges.append([source_index, target_index, 1])  # get the positive edge
    negative_target_index = unsupervised_samples.np_random.choice(
        node_data.index.tolist(), size=1
    )  # generate negative node
    in_sample_edges.append(
        [source_index, negative_target_index[0], 0]
    )  # get the negative edge

for iii in range(len(out_of_sample_edges_df)):
    source_index = out_of_sample_edges_df['source'][iii]
    target_index = out_of_sample_edges_df['target'][iii]
    if source_index > target_index:  # neglect edge direction for the undirected graph
        continue
    out_of_sample_edges.append([source_index, target_index, 1])  # get the positive edge
    negative_target_index = unsupervised_samples.np_random.choice(
        node_data.index.tolist(), size=1
    )  # generate negative node
    out_of_sample_edges.append(
        [source_index, negative_target_index[0], 0]
    )  # get the negative edge

in_sample_edges = np.array(in_sample_edges)
out_of_sample_edges = np.array(out_of_sample_edges)



Construct the edge features from the learned node representations with l2 normed difference, where edge features are the element-wise square of the difference between the embeddings of two head nodes. Other strategy like element-wise product can also be used to construct edge features.

In [53]:
in_sample_edge_feat_from_emb = (
    node_embeddings[in_sample_edges[:, 0]] - node_embeddings[in_sample_edges[:, 1]]
) ** 2
out_of_sample_edge_feat_from_emb = (
    node_embeddings[out_of_sample_edges[:, 0]]
    - node_embeddings[out_of_sample_edges[:, 1]]
) ** 2

Train the Logistic Regression classifier from in-sample edges with the edge features constructed from attri2vec embeddings. 

In [54]:
clf_edge_pred_from_emb = LogisticRegression(
    verbose=0, solver='lbfgs', multi_class='auto', max_iter=500
)
clf_edge_pred_from_emb.fit(in_sample_edge_feat_from_emb, in_sample_edges[:, 2])

LogisticRegression(max_iter=500)

Predict the edge existence probability with the trained Logistic Regression classifier.

In [55]:
edge_pred_from_emb = clf_edge_pred_from_emb.predict_proba(
    out_of_sample_edge_feat_from_emb
)

Get the positive class index of `edge_pred_from_emb`.

In [56]:
if clf_edge_pred_from_emb.classes_[0] == 1:
    positive_class_index = 0
else:
    positive_class_index = 1

Evaluate the AUC score for the prediction with attri2vec embeddings.

In [57]:
roc_auc_score(out_of_sample_edges[:, 2], edge_pred_from_emb[:, positive_class_index])

0.8759709028705389

As the baseline, we also investigate the performance of node content features in predicting the edges of out-of-sample nodes. Firstly, we construct edge features from node content features with the same strategy.

In [58]:
in_sample_edge_rep_from_feat = (
    node_data[feature_names].values[in_sample_edges[:, 0]]
    - node_data[feature_names].values[in_sample_edges[:, 1]]
) ** 2
out_of_sample_edge_rep_from_feat = (
    node_data[feature_names].values[out_of_sample_edges[:, 0]]
    - node_data[feature_names].values[out_of_sample_edges[:, 1]]
) ** 2

Then we train the Logistic Regression classifier from in-sample edges with the edge features constructed from node content features.

In [59]:
clf_edge_pred_from_feat = LogisticRegression(
    verbose=0, solver='lbfgs', multi_class='auto', max_iter=500
)
clf_edge_pred_from_feat.fit(in_sample_edge_rep_from_feat, in_sample_edges[:, 2])

LogisticRegression(max_iter=500)

Predict the edge existence probability with the trained Logistic Regression classifier.

In [60]:
edge_pred_from_feat = clf_edge_pred_from_feat.predict_proba(
    out_of_sample_edge_rep_from_feat
)

Get positive class index of `clf_edge_pred_from_feat`.

In [61]:
if clf_edge_pred_from_feat.classes_[0] == 1:
    positive_class_index = 0
else:
    positive_class_index = 1

Evaluate the AUC score for the prediction with node content features.

In [62]:
roc_auc_score(out_of_sample_edges[:, 2], edge_pred_from_feat[:, positive_class_index])

0.8383823819150906

attri2vec can inductively infer the representations of out-of-sample nodes from their content attributes. As the inferred node representations well capture both structure and node content information, they perform much better than node content features in predicting the links of out-of-sample nodes.

Generate node embeddings for each review

In [63]:
unstructured_reviews_df = pd.read_table(
                          os.path.join(unstructured_reviews_dir, 'review_Video_Games.txt'),
                          sep='\n',
                          header=None,
                          names=['reviewText']
)
unstructured_reviews_df['overall'] = 1
unstructured_reviews_df.iloc[0:400]['overall'] = 5

unstructured_reviews_df['nodes'] = unstructured_reviews_df.apply(lambda x: [node_index for node_index in range(len(source_list)) if source_list[node_index] in x['reviewText'].casefold().replace(',', '')], axis=1)
unstructured_reviews_df['embeddings'] = unstructured_reviews_df.apply(lambda x: np.mean(node_embeddings[x['nodes']], axis=0), axis=1)

# Dropping reviews without node match
unstructured_reviews_df = unstructured_reviews_df.loc[unstructured_reviews_df['nodes'].str.len() > 0].reset_index(drop=True)

unstructured_reviews_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,reviewText,overall,nodes,embeddings
0,My son loves it.,5,"[20, 528, 593, 3690, 3923, 4433, 4557, 4620, 5...","[0.36760202, 0.23434359, 0.22079933, 0.2528594..."
1,Great game. Too bad its only on ps4.,5,"[20, 593, 733, 4319, 4433, 4477, 4497, 4550, 4...","[0.59300476, 0.27829322, 0.29916158, 0.3410903..."
2,"Must have. Extends range of motion, meaning yo...",5,"[77, 518, 593, 1531, 3246, 3512, 4261, 4262, 4...","[0.42883372, 0.2780619, 0.23285682, 0.26259598..."
3,"Ive been playing for a few hours, and its quit...",5,"[20, 593, 686, 1531, 1607, 3128, 3512, 4254, 4...","[0.44911447, 0.24518844, 0.26324612, 0.2924172..."
4,One of the best gifts ibought my son. He loves...,5,"[20, 528, 593, 1791, 1910, 3512, 3690, 4250, 4...","[0.5149061, 0.24813408, 0.2519347, 0.20872942,..."
...,...,...,...,...
795,S didnt work u lucky i aknt send it back. Sold...,1,"[5, 8, 9, 20, 528, 593, 1531, 3512, 4274, 4278...","[0.42743805, 0.21293792, 0.23990469, 0.2746062..."
796,Loses signal often. Switched back to wired mouse.,1,"[1, 2, 4, 20, 593, 3512, 4271, 4272, 4273, 427...","[0.36829168, 0.27692646, 0.29766253, 0.2676395..."
797,Its nhl 14 with bergeron on the box.,1,"[20, 593, 3512, 4267, 4268, 4269, 4270, 4433, ...","[0.43893495, 0.38454327, 0.312465, 0.34976432,..."
798,"Bad, reply to customer service no one anwswer.",1,"[593, 2901, 4322, 4433, 4497, 4598, 4620, 4994...","[0.51745546, 0.35617092, 0.38024682, 0.4323207..."


Get average number of words of each review

In [64]:
def n_words(text):
  aux_list = text.replace('.', '').split(' ')
  return len(aux_list)

avg_words = np.mean(unstructured_reviews_df.apply(lambda x: n_words(x['reviewText']), axis=1))
avg_words

18.59625

## Apply SVM classifier to selected reviews

Establish train and test sets

In [65]:
dict_train = {}
dict_test = {}
for i in range(10):
  dict_train[i] = unstructured_reviews_df.sample(frac=0.75)
  dict_test[i] = pd.concat([unstructured_reviews_df, dict_train[i]]).drop_duplicates(subset='reviewText', keep=False)

Initialize model for multiple tests

In [66]:
model_eval = pd.DataFrame(columns=['Model', 'Macro f1', 'Macro precision', 'Macro recall', 'Weighted f1', 'Weighted precision', 'Weighted recall'])

for i in range(10):
  X_train = pd.DataFrame(dict_train[i]['embeddings'].to_list())
  X_test = pd.DataFrame(dict_test[i]['embeddings'].to_list())
  y_train = dict_train[i]['overall']
  y_train = [0 if i==1 else 1 for i in y_train]
  y_test = dict_test[i]['overall']
  y_test = [0 if i==1 else 1 for i in y_test]

  clf = make_pipeline(StandardScaler(), SVC(C=3, kernel='rbf', gamma='auto'))
  clf.fit(X_train, y_train)
  y_pred_train = clf.predict(X_train)
  y_pred_test = clf.predict(X_test)

  model_eval = model_eval.append({'Model': i,
                                    'Macro f1': f1_score(y_test, y_pred_test, average='macro'), 
                                    'Macro precision': precision_score(y_test, y_pred_test, average='macro'), 
                                    'Macro recall': recall_score(y_test, y_pred_test, average='macro'), 
                                    'Weighted f1': f1_score(y_test, y_pred_test, average='weighted'), 
                                    'Weighted precision': precision_score(y_test, y_pred_test, average='weighted'), 
                                    'Weighted recall': recall_score(y_test, y_pred_test, average='weighted')
                                    }, ignore_index=True)
  
model_eval

,Model,Macro f1,Macro precision,Macro recall,Weighted f1,Weighted precision,Weighted recall
0,0.0,0.707071,0.708227,0.708227,0.707071,0.709383,0.707071
1,1.0,0.734675,0.735000,0.736157,0.735325,0.737450,0.735000
2,2.0,0.709535,0.713131,0.711084,0.709303,0.713758,0.710000
3,3.0,0.764853,0.765224,0.764826,0.764912,0.765168,0.765000
4,4.0,0.719888,0.721249,0.722138,0.720280,0.724164,0.720000
5,5.0,0.723367,0.726343,0.724833,0.723157,0.727145,0.723618
6,6.0,0.652001,0.652880,0.651952,0.652739,0.653039,0.653266
7,7.0,0.753769,0.753940,0.753940,0.753769,0.754112,0.753769
8,8.0,0.734940,0.734923,0.734994,0.735020,0.735077,0.735000
9,9.0,0.687492,0.694515,0.686612,0.691851,0.694713,0.695000


In [67]:
model_eval.describe()

,Model,Macro f1,Macro precision,Macro recall,Weighted f1,Weighted precision,Weighted recall
count,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,4.50000,0.718759,0.720543,0.719476,0.719343,0.721401,0.719772
std,3.02765,0.032564,0.031665,0.032721,0.032010,0.031727,0.031576
min,0.00000,0.652001,0.652880,0.651952,0.652739,0.653039,0.653266
25%,2.25000,0.707687,0.709453,0.708941,0.707629,0.710477,0.707803
50%,4.50000,0.721627,0.723796,0.723486,0.721719,0.725654,0.721809
75%,6.75000,0.734874,0.734981,0.735866,0.735249,0.736857,0.735000
max,9.00000,0.764853,0.765224,0.764826,0.764912,0.765168,0.765000


# Model analysis with bag-of-words

Applying CountVectorizer() to get features vectorization

In [68]:
docs = unstructured_reviews_df['reviewText'].to_list()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs)
vectorizer.get_feature_names_out()

vectors = X.toarray()

feature_names = ['w_{}'.format(ii) for ii in range(len(vectors[0]))]
bow_node_data = pd.DataFrame(vectors, columns=feature_names)

display(bow_node_data.head())

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_2529,w_2530,w_2531,w_2532,w_2533,w_2534,w_2535,w_2536,w_2537,w_2538
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##Apply SVM classifier to selected reviews

Initialize model for multiple tests

In [69]:
model_eval = pd.DataFrame(columns=['Model', 'Macro f1', 'Macro precision', 'Macro recall', 'Weighted f1', 'Weighted precision', 'Weighted recall'])

for i in range(10):

  X_train = bow_node_data.iloc[dict_train[i].index]
  X_test = bow_node_data.iloc[dict_test[i].index]
  y_train = dict_train[i]['overall']
  y_train = [0 if i==1 else 1 for i in y_train]
  y_test = dict_test[i]['overall']
  y_test = [0 if i==1 else 1 for i in y_test]

  clf = make_pipeline(StandardScaler(), SVC(C=3, kernel='rbf', gamma='auto'))
  clf.fit(X_train, y_train)
  y_pred_train = clf.predict(X_train)
  y_pred_test = clf.predict(X_test)

  model_eval = model_eval.append({'Model': i,
                                    'Macro f1': f1_score(y_test, y_pred_test, average='macro'), 
                                    'Macro precision': precision_score(y_test, y_pred_test, average='macro'), 
                                    'Macro recall': recall_score(y_test, y_pred_test, average='macro'), 
                                    'Weighted f1': f1_score(y_test, y_pred_test, average='weighted'), 
                                    'Weighted precision': precision_score(y_test, y_pred_test, average='weighted'), 
                                    'Weighted recall': recall_score(y_test, y_pred_test, average='weighted')
                                    }, ignore_index=True)
  
model_eval

,Model,Macro f1,Macro precision,Macro recall,Weighted f1,Weighted precision,Weighted recall
0,0.0,0.827650,0.840440,0.832090,0.827228,0.843479,0.828283
1,1.0,0.821242,0.833438,0.818862,0.823056,0.830753,0.825000
2,2.0,0.839422,0.842424,0.839136,0.839615,0.841939,0.840000
3,3.0,0.773363,0.781404,0.774227,0.773555,0.781004,0.775000
4,4.0,0.814884,0.823864,0.820772,0.814560,0.829034,0.815000
5,5.0,0.823675,0.825020,0.823378,0.823898,0.824720,0.824121
6,6.0,0.778844,0.781244,0.780390,0.778727,0.782509,0.778894
7,7.0,0.878880,0.883651,0.878612,0.878999,0.883096,0.879397
8,8.0,0.849458,0.852525,0.849140,0.849639,0.852020,0.850000
9,9.0,0.759784,0.772890,0.768928,0.759135,0.780624,0.760000


In [70]:
model_eval.describe()

,Model,Macro f1,Macro precision,Macro recall,Weighted f1,Weighted precision,Weighted recall
count,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,4.50000,0.816720,0.823690,0.818554,0.816841,0.824918,0.817569
std,3.02765,0.036816,0.035474,0.035058,0.036992,0.034146,0.036847
min,0.00000,0.759784,0.772890,0.768928,0.759135,0.780624,0.760000
25%,2.25000,0.787854,0.792019,0.790008,0.787685,0.793062,0.787921
50%,4.50000,0.822458,0.829229,0.822075,0.823477,0.829894,0.824560
75%,6.75000,0.836479,0.841928,0.837374,0.836518,0.843094,0.837071
max,9.00000,0.878880,0.883651,0.878612,0.878999,0.883096,0.879397
